In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r', encoding='utf-8').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos.keys())
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [33]:
block_size = 3 # Offset of context

def build_dataset(words):
    X, Y = [], []
    for w in words:
        #print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '-->', itos[ix])
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

In [43]:
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate {str(app):5s} | maxdiff: {maxdiff}')

In [35]:
n_embd = 10
n_hidden = 64

# Network layers
g = torch.Generator().manual_seed(21474783647)
C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * 0.2
b1 = torch.randn(n_hidden, generator=g) * 0.1 # using b1 just for fun, it's useless anyways ...
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.01
b2 = torch.randn(vocab_size, generator=g) * 0.1

# Batchnorm. parameters
bngain = torch.ones((1, n_hidden))*0.1 + 1.0
bnbias = torch.zeros((1, n_hidden))*0.1

parameters = [C, W1, W2, b1, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

4137


In [36]:
batch_size = 32
n = batch_size
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

The very part-wise procedure for the forward pass below is so we can then calculate the backward pass
on it during this exercise:

In [37]:
# FORWARD PASS
# Emb layer
emb = C[Xb] # 32, 3, 2
emb_cat = emb.view(emb.shape[0], -1) # Concat vectors

# Linear layer 1
hprebn = emb_cat @ W1 # + b1 # Hidden layer preactivation

# Batch-Normalization Layer
bnmeani = 1/n  *hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff ** 2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5) ** -0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias


# Activation FN, Non-Linearity
h = torch.tanh(hpreact) # 32, 100

# Linear Layer 2
logits = h @ W2 + b2 # 32, 27

# Cross Entropy Loss (same as F.cross_entropy)
logits_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logits_maxes # 0 Center for numerical stability, since we exponentiate the logits
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# BACKWARD PASS
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logits_maxes, logits, h, hpreact, bnraw,
          bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
          emb_cat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.2892, grad_fn=<NegBackward0>)

### Exercise 1: Calculate the backprop of the loss gradient explicitly

In [49]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n

# Derivative of logprobs with respect to probs + chain-rule applied
dprobs = (1.0 / probs) * dlogprobs

# Derivative of probs with respect to counts_sum_inv, Note: care for the broadcasting operation required for the matmul here!
# the sum operation collapses the first dimension (columns), which means we get the row sums here!
dcounts_sum_inv = (counts * dprobs).sum(1, keepdims=True)

# Not additional summation required here, since the shapes are fine when broadcasted anyway (since it broadcasts row-wise now, not col-wise)
# Care that counts is used twice: once by probs and once by counts_sum and counts_sum_inv
dcounts = counts_sum_inv * dprobs

dcounts_sum = (-1.0 / counts_sum ** 2) * dcounts_sum_inv

# Broadcasting here implements the replication for us internally
# The intuition behind this is, that a sum operation basically distributes the gradient (e.g. here dcounts_sum) to all of the elements
# that contributed to it!
dcounts += torch.ones_like(counts_sum) * dcounts_sum

# Could also use counts * dcounts since counts = norm_logits.exp()
dnorm_logits = norm_logits.exp() * dcounts

# Is the same as dnorm_logits since we again get 1s for all the elements that contribute, Note: This is not the final dlogits, as it
# has another branch that flows into it!
dlogits = dnorm_logits.clone()
# Is the negative of dnorm_logits for the same reason, but we have to collapse the column dim to get the same shape as maxes again (32,1)
dlogits_maxes = (-dnorm_logits).sum(1, keepdims=True)

logit_max_mask = torch.zeros_like(logits)
logit_max_mask[range(n),  logits.max(1).indices]  = 1
dlogits += logit_max_mask * dlogits_maxes

dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)

dhpreact = (1.0 - h ** 2) * dh

dbngain = (bnraw * dhpreact).sum(0, keepdims=True)
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(0, keepdim=True)

# This bndiff is not complete, as there is another branch missing that we will backprop at the end!!
dbndiff = bnvar_inv * dbnraw
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)

dbnvar = (-0.5 * (bnvar + 1e-5) ** (-1.5)) * dbnvar_inv

dbndiff2 = (1/(n-1)) * dbnvar
# bndiff2 = bndiff ** 2
dbndiff += 2 * bndiff * dbndiff2

# Carefull with the shapes here! hprebn is shape (32,64) while bnmeani is (1,64)
# -> This means in the forward pass there is broadcasting, which means in the backward pass
# we will have to do a sum operation for the bnmeani!
dbnmeani = (-dbndiff).sum(0, keepdims=True)
# Note: hprebn is not complete, since it has two branches, one that depends on dbnmeani!!
dhprebn = dbndiff.clone()
#bnmeani = 1/n  *hprebn.sum(0, keepdim=True)
dhprebn += (1/n) * dbnmeani

dembcat = dhprebn @ W1.T
dW1 = emb_cat.T @ dhprebn
db1 = dhprebn.sum(0)

demb = dembcat.view((32, 3, 10))
dC = torch.zeros_like(C)
for i in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[i, j]
        dC[ix] += demb[i, j]

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logits_maxes', dlogits_maxes, logits_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnraw', dbnraw, bnraw)
cmp('bnbias', dbnbias, bnbias)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('emb_cat', dembcat, emb_cat)
cmp('W1', dW1, W1)
#cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate True  | maxdiff: 0.0
probs           | exact: True  | approximate True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate True  | maxdiff: 0.0
counts          | exact: True  | approximate True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate True  | maxdiff: 0.0
logits_maxes    | exact: True  | approximate True  | maxdiff: 0.0
logits          | exact: True  | approximate True  | maxdiff: 0.0
h               | exact: True  | approximate True  | maxdiff: 0.0
W2              | exact: True  | approximate True  | maxdiff: 0.0
b2              | exact: True  | approximate True  | maxdiff: 0.0
hpreact         | exact: True  | approximate True  | maxdiff: 0.0
bngain          | exact: True  | approximate True  | maxdiff: 0.0
bnraw           | exact: True  | approximate True  | maxdiff: 0.0
bnbias          | exact: True  | approximate True  | maxdiff: 0.0
bnvar_inv 

## Exercise 2: Cross Entropy Loss Backward Pass